# 107: Swissmetro Latent Class

In [ ]:
# TEST
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", 999)
pd.set_option("expand_frame_repr", False)
pd.set_option("display.precision", 3)
np.set_printoptions(precision=12)
from pytest import approx

import larch as lx

In [ ]:
import numpy as np
import pandas as pd

import larch as lx
from larch import P, X

print(lx.versions())

This example is a latent class mode choice model (also known as a discrete mixture model) built using the Swissmetro example dataset.
We will follow the formulation of this model from the Biogeme documentation on the
[equivalent model](https://biogeme.epfl.ch/sphinx/auto_examples/swissmetro/plot_b07discrete_mixture.html).

The first step is to load the data and do some pre-processing, so that the format and scale of the 
data matches that from the Biogeme example.

In [ ]:
raw = pd.read_csv(lx.example_file("swissmetro.csv.gz"))
raw["SM_COST"] = raw["SM_CO"] * (raw["GA"] == 0)
raw["TRAIN_COST"] = raw.eval("TRAIN_CO * (GA == 0)")
raw["TRAIN_COST_SCALED"] = raw["TRAIN_COST"] / 100
raw["TRAIN_TT_SCALED"] = raw["TRAIN_TT"] / 100
raw["SM_COST_SCALED"] = raw.eval("SM_COST / 100")
raw["SM_TT_SCALED"] = raw["SM_TT"] / 100
raw["CAR_CO_SCALED"] = raw["CAR_CO"] / 100
raw["CAR_TT_SCALED"] = raw["CAR_TT"] / 100
raw["CAR_AV_SP"] = raw.eval("CAR_AV * (SP!=0)")
raw["TRAIN_AV_SP"] = raw.eval("TRAIN_AV * (SP!=0)")

We now have a pandas DataFrame in "wide" or [idco](idco) format, with one row per choice observation.

In [ ]:
raw

We can convert this to a Larch Dataset, filtering to include only the cases of interest,
following the same filter applied in the Biogeme example.

In [ ]:
data = lx.Dataset.construct.from_idco(raw).dc.query_cases(
    "PURPOSE in (1,3) and CHOICE != 0"
)

The result is a Dataset with 6,768 cases.

In [ ]:
data

For convenience, we can collect the names of the variables that define alternative availability
into a dictionary, keyed on the codes we will use to represent each alternative.

In [ ]:
availability_co_vars = {
    1: "TRAIN_AV_SP",
    2: "SM_AV",
    3: "CAR_AV_SP",
}

Then we can contruct choice models for each of the classes.  This is done in the
usual manner for Larch choice models, assigning utility functions, alternative 
availability conditions, and choice markers in the usual manner.

In this example, Class 1 chooses based on cost, and the utility functions include
a set of alternative specific constants.

In [ ]:
m1 = lx.Model(title="Model1")
m1.utility_co[1] = P("ASC_TRAIN") + X("TRAIN_COST_SCALED") * P("B_COST")
m1.utility_co[2] = X("SM_COST_SCALED") * P("B_COST")
m1.utility_co[3] = P("ASC_CAR") + X("CAR_CO_SCALED") * P("B_COST")
m1.availability_co_vars = availability_co_vars
m1.choice_co_code = "CHOICE"

Class 2 is the nearly same model, defined completely seperately but in a similar manner. 
The only difference is in the utility functions, which adds travel time to the utility.

In [ ]:
m2 = lx.Model(title="Model2")
m2.utility_co[1] = (
    P("ASC_TRAIN")
    + X("TRAIN_TT_SCALED") * P("B_TIME")
    + X("TRAIN_COST_SCALED") * P("B_COST")
)
m2.utility_co[2] = X("SM_TT_SCALED") * P("B_TIME") + X("SM_COST_SCALED") * P("B_COST")
m2.utility_co[3] = (
    P("ASC_CAR") + X("CAR_TT_SCALED") * P("B_TIME") + X("CAR_CO_SCALED") * P("B_COST")
)
m2.availability_co_vars = availability_co_vars
m2.choice_co_code = "CHOICE"

The class membership model is another Larch choice model.  This model is 
quite simple, as it does not need alternative availability conditions
nor choice indicators.  The "alternatives" in this model are the 
class choice models we defined above, and we can assign them unique
identifyong codes for clarity.

For this latent class model, the definition is quite simple, just constants 
for each class.  As typical for logit models, one "alternative" (here, a class)
is omitted to be the reference point.  It is possible to use chooser attributes
to inform the relative likelihood of class membership, but that is not what
we will do here.

In [ ]:
mk = lx.Model()
mk.utility_co[102] = P("W")

Finally, we assemble all the component models into one `LatentClass` structure.

In [ ]:
b = lx.LatentClass(
    mk,
    {101: m1, 102: m2},
    datatree=data.dc.set_altids([1, 2, 3]),
)

Similar to other models, we can set a parameter cap to improve estimation stability.

In [ ]:
b.set_cap(25)

In [ ]:
# TEST
assert b.loglike() == approx(-6964.6640625, rel=1e-4)

In [ ]:
# TEST
assert b.d_loglike() == approx(
    [-99.00003, -1541.5, -224.60551, -923.5081, 0.0],
    rel=1e-5,
)

In [ ]:
b.d_loglike()

In [ ]:
result = b.maximize_loglike(method="slsqp")

In [ ]:
b.calculate_parameter_covariance();

In [ ]:
b.parameter_summary()

In [ ]:
# TEST
assert result.loglike == approx(-5208.498, rel=1e-5)
assert b.pstderr == approx(
    np.array([0.05049792, 0.06086081, 0.06121413, 0.17560391, 0.11610422]), rel=5e-3
)
assert b.pvals == approx(
    np.array(
        [
            0.12573555,
            -0.39753284,
            -1.26550162,
            -2.80171093,
            1.09212252,
        ]
    ),
    rel=5e-3,
)